In [ ]:
import transformer
from transformers import GPT2Tokenizer
tok = transformers.GPT2Tokenizer.from_pretrained('gpt2')

: 

In [ ]:
lotr_sent = 'Aragorn told Frodo to mind Lothlorien'
tok.convert_ids_to_tokens(tok(lotr_sent)['input_ids'])
['Ar', 'ag', 'orn', 'Ġtold', 'ĠFro', 'do', 'Ġto', 'Ġmind', 'ĠL', 'oth', 'lor', 'ien']

: 

In [ ]:
lotr_sent = 'Aragorn told Frodo to mind Lothlorien'
model = transformers.AutoModelForCausalLM.from_pretrained('gpt2')
tok.decode(model.generate(**tok(lotr_sent, return_tensors='pt'), do_sample=True)[0])
'Aragorn told Frodo to mind Lothlorien for a while.\n\nThe Ring'

In [ ]:
tok.add_tokens(['Aragorn', 'Frodo', 'Lothlorien'])
tok.convert_ids_to_tokens(tok(lotr_sent)['input_ids'])
['Aragorn', 'told', 'Frodo', 'to', 'Ġmind', 'Lothlorien']

In [ ]:
model.resize_token_embeddings(len(tok))

: 

In [ ]:
sent2 = 'Dogs are great because they are '
tok.decode(model.generate(**tok(sent2, return_tensors='pt'), do_sample=True)[0])
'Dogs are great because they are  Aragorn Aragorn Aragorn Aragorn Frodo Aragorn'

sanity check + new implementation

In [ ]:
tok = transformers.GPT2Tokenizer.from_pretrained('gpt2')
model = transformers.AutoModelForCausalLM.from_pretrained('gpt2')
tok.add_tokens(['Aragorn', 'Frodo', 'Lothlorien'])
model.resize_token_embeddings(len(tok))

In [ ]:
import torch
params = model.state_dict()
embeddings = params['transformer.wte.weight']
pre_expansion_embeddings = embeddings[:-3,:]
mu = torch.mean(pre_expansion_embeddings, dim=0)
n = pre_expansion_embeddings.size()[0]
sigma = ((pre_expansion_embeddings - mu).T @ (pre_expansion_embeddings - mu)) / n
dist = torch.distributions.multivariate_normal.MultivariateNormal(
        mu, covariance_matrix=1e-5*sigma)

In [ ]:
new_embeddings = torch.stack(tuple((dist.sample() for _ in range(3))), dim=0)
embeddings[-3:,:] = new_embeddings
params['transformer.wte.weight'][-3:,:] = new_embeddings
model.load_state_dict(params)

In [ ]:
sent2 = 'Dogs are great because they are '
tok.decode(model.generate(**tok(sent2, return_tensors='pt'), do_sample=True)[0])
"Dogs are great because they are icky and don't tend to get in the way much,"